In [2]:
import pandas as pd
import requests, os

In [4]:
sf_df = pd.read_excel(r'D:\ODOC\jupyter_notebook\제품용기\product_fulldata_random.xlsx')
sdf = sf_df[['제품명','제품이미지']]
sdf.head(3)

,제품명,제품이미지
0,바이오힐보 프로바이오덤 리프팅 크림 (본품50ml+앰플7ml+7ml 증정),https://image.oliveyoung.co.kr/uploads/images/...
1,한자플라스트 후레쉬 액티브 풋 데오도란트 150ml,https://image.oliveyoung.co.kr/uploads/images/...
2,닥터자르트 세라마이딘 핸드크림 50ml,https://image.oliveyoung.co.kr/uploads/images/...


#### 학습셋 구성

In [91]:
# 이미지 로컬로 다운
for index, row in sdf[0:1000].iterrows():
    try:
        sdf.at[index, 'images'] = f'/product_img_{row.P_ID}.jpg'
        image_name = f"D:/ODOC/jupyter_notebook/제품용기/product_images/product_img_{row['P_ID']}.jpg"
        image_url = row['제품이미지']
        
        response = requests.get(image_url)
        with open(image_name, 'wb') as f:
            f.write(response.content)
    except:   #이미지가 없거나 유효하지 않은 이미지 예외처리
        pass

In [ ]:
#로보플로우를 이용해 yolo학습

#색상은 안 중요하니 전부 흑백 처리(grayscale)
#회전, 뒤집기 -> 3배 증식 적용

In [47]:
#학습 제외 사진

#제품사진이 아닌 증정품(파우치, 가방 등) 사진인 경우
#사진 속에 여러 화장품이 있는 경우(세트)
#화장품이 아닌 경우(면도기, 마사기지, 고데기 등)
#실 제품사진이 아니라 도안만 나온 경우(평면 그림)
#포장용기 없이 실 제품 사진만 있는 경우(비누)

#### 테스트 셋 구성

In [34]:
#필요없는 제품유형은 삭제 (실제 프로그램에서 사용하지 않을 제품들)
cat_remove = ['마스크/팩','더모 코스메틱','아이 메이크업','베이스 메이크업']
sdf = sdf[~sdf['카테고리_1'].isin(cat_remove)]
cat_remove = ['선물세트','제모용품','향수/매너용품','쉐이빙','필링/패드','립케어',
'메이크업','워시/스크럽','마스크팩','태닝/애프터선','데오도란트','염색약/펌']
sdf = sdf[~sdf['카테고리_2'].isin(cat_remove)]
cat_remove = ['염색/다운펌/기기','토너/로션/세트','선쿠션/파우더', '스킨케어 세트', 
       '클렌징 비누','발/다리피로회복','발각질제거', '핸드마스크/세니타이저']
sdf = sdf[~sdf['카테고리_3'].isin(cat_remove)]

#세트제품 삭제(한 사진에 여러 제품 있음)
#스틱제품 삭제
idx_tf = sdf['제품명'].str.contains('세트|스틱|파일|티슈|리필|패드|마스크|기저귀|디퓨저|패치|생리대', na=False)
sdf = sdf[~idx_tf]

sdf.reset_index(drop=True, inplace=True)

In [88]:
sdf.to_excel(r'D:\ODOC\jupyter_notebook\제품용기\random_filltered.xlsx', index=False)
sdf = pd.read_excel(r'D:\ODOC\jupyter_notebook\제품용기\random_filltered.xlsx')

In [147]:
# 모델에 학습시킨 1000장 제외+필요없는 행 삭제-> 3958개만 사용
sdf_test = sdf[1000:]
cat_remove = ['염색/다운펌/기기','토너/로션/세트','선쿠션/파우더', '스킨케어 세트', 
    '클렌징 비누','발/다리피로회복','발각질제거', '핸드마스크/세니타이저','발냄새제거',
    '퍼퓸바디미스트','기타','스타일링','풋크림/보습용품','선스틱/선스프레이','데오/바디미스트']
sdf_test = sdf_test[~sdf_test['카테고리_3'].isin(cat_remove)]

idx_tf = sdf_test['제품명'].str.contains('파우더', na=False)
idx_cat3 = sdf_test['카테고리_3']=='바디오일/바디파우더'
sdf_test = sdf_test[~(idx_tf&idx_cat3)]

idx_tf = sdf_test['제품명'].str.contains('비누|생리대|쿠션|립밤', na=False)
sdf_test = sdf_test[~idx_tf]

sdf_test.reset_index(drop=True, inplace=True)
sdf_test.to_excel(r'D:\ODOC\jupyter_notebook\제품용기\random_filltered_test.xlsx', index=False)
sdf_test = pd.read_excel(r'D:\ODOC\jupyter_notebook\제품용기\random_filltered_test.xlsx')

In [1]:
#저장한 모델 불러오기

# !pip install PyYaml
# !pip install ultralytics
import ultralytics
ultralytics.checks()    # 욜로 v8를 쓸 수 있는 버전인 지 확인해주는 것
from ultralytics import YOLO
import yaml, zipfile, os, shutil
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import pandas as pd
import requests, os

In [ ]:
path = "D:\ODOC\jupyter_notebook\제품용기/beauty_case_best.pt"
model = YOLO(path)

In [4]:
fdf = pd.read_excel(r'D:\ODOC\jupyter_notebook\제품용기\product_fulldata.xlsx')
fdf.drop('Unnamed: 0', axis=1, inplace=True)
fdf.head(2)

In [ ]:
#이미지 다운로드
base_dir = r'D:\ODOC\jupyter_notebook\제품용기\images_full'
for index, row in fdf.iterrows():
    try:
        image_name = f"product_img_{row['P_ID']}.jpg"
        image_path = os.path.join(base_dir, image_name)
        image_url = row['제품이미지']

        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)

        fdf.at[index, 'images'] = image_name
    except Exception as e:
        print(f"Error downloading image for P_ID {row['P_ID']}: {e}")

In [298]:
#객체탐지 수행
image_folder = 'D:\\ODOC\\jupyter_notebook\\제품용기\\images_full\\'

for i in range(2000, len(fdf), 100):
    chunk = fdf.iloc[i:i+100]
    for idx,row in chunk.iterrows():
        try:
            predict = model.predict(os.path.join(image_folder, row['images']))
            fdf.at[idx, 'predictions'] = predict[0].boxes
        except Exception as e:
            print(f"{e}: {row['images']}")
    fdf.to_excel(r'D:\ODOC\jupyter_notebook\제품용기\product_fulldata.xlsx', index=False)


image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6887.jpg: 416x416 1 bottle, 818.3ms
Speed: 5.0ms preprocess, 818.3ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6888.jpg: 416x416 1 bottle, 750.5ms
Speed: 3.0ms preprocess, 750.5ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6889.jpg: 416x416 1 bottle, 831.2ms
Speed: 19.9ms preprocess, 831.2ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6890.jpg: 416x416 1 bottle, 665.8ms
Speed: 5.0ms preprocess, 665.8ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6891.jpg: 416x416 1 bottle, 693.7ms
Speed: 7.0ms preprocess, 693.7ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

ima


image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6928.jpg: 416x416 1 tube, 653.8ms
Speed: 22.9ms preprocess, 653.8ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6929.jpg: 416x416 1 tube, 547.2ms
Speed: 3.0ms preprocess, 547.2ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6930.jpg: 416x416 1 tube, 829.2ms
Speed: 3.0ms preprocess, 829.2ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6931.jpg: 416x416 1 tube, 816.3ms
Speed: 2.0ms preprocess, 816.3ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6932.jpg: 416x416 1 bottle, 1016.6ms
Speed: 3.0ms preprocess, 1016.6ms inference, 8.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1


image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6969.jpg: 416x416 1 bottle, 522.3ms
Speed: 2.0ms preprocess, 522.3ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6970.jpg: 416x416 1 bottle, 532.2ms
Speed: 1.0ms preprocess, 532.2ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6971.jpg: 416x416 1 bottle, 490.4ms
Speed: 2.0ms preprocess, 490.4ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6972.jpg: 416x416 1 bottle, 2 tubes, 483.4ms
Speed: 2.0ms preprocess, 483.4ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6973.jpg: 416x416 1 bottle, 2 tubes, 475.4ms
Speed: 5.0ms preprocess, 475.4ms inference, 1.0ms postprocess per image at shape (1, 


image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6837.jpg: 416x416 1 bottle, 423.6ms
Speed: 2.0ms preprocess, 423.6ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6838.jpg: 416x416 2 bottles, 1 jar, 1 tube, 403.8ms
Speed: 2.0ms preprocess, 403.8ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6839.jpg: 416x416 1 bottle, 448.5ms
Speed: 2.0ms preprocess, 448.5ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6840.jpg: 416x416 1 tube, 386.7ms
Speed: 1.0ms preprocess, 386.7ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6841.jpg: 416x416 1 tube, 411.6ms
Speed: 1.0ms preprocess, 411.6ms inference, 1.0ms postprocess per image at shape (1, 3, 416


image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6792.jpg: 352x416 3 bottles, 789.9ms
Speed: 2.0ms preprocess, 789.9ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6793.jpg: 352x416 3 bottles, 535.7ms
Speed: 2.0ms preprocess, 535.7ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6794.jpg: 352x416 3 bottles, 439.2ms
Speed: 2.0ms preprocess, 439.2ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6795.jpg: 352x416 3 bottles, 571.0ms
Speed: 2.0ms preprocess, 571.0ms inference, 2.0ms postprocess per image at shape (1, 3, 352, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_6796.jpg: 416x416 1 bottle, 1140.7ms
Speed: 8.0ms preprocess, 1140.7ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

...


image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_169.jpg: 416x416 1 tube, 635.5ms
Speed: 3.0ms preprocess, 635.5ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_170.jpg: 416x416 4 bottles, 626.0ms
Speed: 3.0ms preprocess, 626.0ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_171.jpg: 416x416 (no detections), 578.1ms
Speed: 4.0ms preprocess, 578.1ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_172.jpg: 416x416 1 spoide, 599.4ms
Speed: 2.0ms preprocess, 599.4ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_173.jpg: 416x416 1 bottle, 545.1ms
Speed: 2.0ms preprocess, 545.1ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

ima


image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_69.jpg: 416x416 1 bottle, 1 jar, 630.5ms
Speed: 2.0ms preprocess, 630.5ms inference, 3.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_70.jpg: 416x416 1 bottle, 597.2ms
Speed: 3.0ms preprocess, 597.2ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_71.jpg: 416x416 1 bottle, 608.0ms
Speed: 2.0ms preprocess, 608.0ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_72.jpg: 416x416 1 jar, 662.3ms
Speed: 2.0ms preprocess, 662.3ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_73.jpg: 416x416 1 bottle, 648.0ms
Speed: 3.0ms preprocess, 648.0ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 

Speed: 3.0ms preprocess, 626.9ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_111.jpg: 416x416 1 bottle, 643.4ms
Speed: 2.0ms preprocess, 643.4ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_112.jpg: 416x416 1 bottle, 687.7ms
Speed: 6.0ms preprocess, 687.7ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_113.jpg: 416x416 2 bottles, 1 jar, 683.7ms
Speed: 3.0ms preprocess, 683.7ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_114.jpg: 416x416 1 tube, 668.3ms
Speed: 2.0ms preprocess, 668.3ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_115.jpg: 416x416 1 bottle, 713.0ms
Spee


image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_25.jpg: 416x416 (no detections), 569.1ms
Speed: 2.0ms preprocess, 569.1ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_26.jpg: 416x416 2 tubes, 593.6ms
Speed: 2.0ms preprocess, 593.6ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_27.jpg: 416x416 2 tubes, 617.3ms
Speed: 3.0ms preprocess, 617.3ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_28.jpg: 416x416 3 tubes, 646.8ms
Speed: 2.0ms preprocess, 646.8ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_29.jpg: 416x416 2 tubes, 627.5ms
Speed: 2.9ms preprocess, 627.5ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D

Speed: 2.0ms preprocess, 564.1ms inference, 6.0ms postprocess per image at shape (1, 3, 416, 352)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_8967.jpg: 416x352 1 bottle, 530.9ms
Speed: 2.0ms preprocess, 530.9ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 352)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_8968.jpg: 416x288 1 bottle, 449.5ms
Speed: 2.0ms preprocess, 449.5ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 288)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_8969.jpg: 416x416 1 tube, 712.6ms
Speed: 4.0ms preprocess, 712.6ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_8970.jpg: 416x416 1 bottle, 633.9ms
Speed: 4.0ms preprocess, 633.9ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 D:\ODOC\jupyter_notebook\제품용기\images_full\product_img_8971.jpg: 416x416 1 jar, 599.0ms
Speed: 2.0

In [317]:
fdf_na = fdf[~fdf.predictions.isna()]
fdf['class'] = None
fdf['class'] = fdf_na.predictions.apply(lambda x: x.cls.tolist())

fdf_na = fdf[~fdf['class'].isna()]
fdf['class'] = fdf_na['class'].apply(lambda x: list(set(x)))

In [338]:
#탐지된 객체가 1종류인 경우만
fdf_na = fdf[~fdf['class'].isna()]
fdf_cls = fdf_na[~(fdf_na['class'].apply(len)>=2)]

In [341]:
# fdf.to_excel(r'D:\ODOC\jupyter_notebook\제품용기\product_fulldata.xlsx', index=False)
# fdf_cls.to_excel(r'D:\ODOC\jupyter_notebook\제품용기\product_oneclass.xlsx', index=False)

In [67]:
#한 사진에 두 개 이상의 클래스 예측시x
idx1 = sdf[(sdf['bottle']==1) & (sdf['spoide']==0) & (sdf['jar']==0) & (sdf['tube']==0)].index
idx2 = sdf[(sdf['bottle']==0) & (sdf['spoide']==1) & (sdf['jar']==0) & (sdf['tube']==0)].index
idx3 = sdf[(sdf['bottle']==0) & (sdf['spoide']==0) & (sdf['jar']==1) & (sdf['tube']==0)].index
idx4 = sdf[(sdf['bottle']==0) & (sdf['spoide']==0) & (sdf['jar']==0) & (sdf['tube']==1)].index
sdf = sdf.iloc[idx1|idx2|idx3|idx4]

In [45]:
#클래스 정리
idx1 = (sdf['bottle']==1) & (sdf['spoide']==0) & (sdf['jar']==0) & (sdf['tube']==0)
idx2 = (sdf['bottle']==0) & (sdf['spoide']==1) & (sdf['jar']==0) & (sdf['tube']==0)
idx3 = (sdf['bottle']==0) & (sdf['spoide']==0) & (sdf['jar']==1) & (sdf['tube']==0)
idx4 = (sdf['bottle']==0) & (sdf['spoide']==0) & (sdf['jar']==0) & (sdf['tube']==1)
sdf.loc[idx1,'class'] = 1   #보틀
sdf.loc[idx2,'class'] = 2   #스포이드
sdf.loc[idx3,'class'] = 3   #포트(jar)
sdf.loc[idx4,'class'] = 4   #튜브

In [ ]:
# 가격 누락 행 삭제
# df[df.가격=='false']
df.drop(1958,inplace=True)

In [47]:
df = sdf[['P_ID','제품명', '카테고리_1', '카테고리_2', '카테고리_3', '브랜드명', '가격',
       '세일_가격', '용량', '제조사', '제조국', '성분', '기능성_심사여부', 'class']]

In [66]:
#가격 텍스트->숫자
import re
df.가격 = df.가격.apply(lambda x: int(re.sub(r'[원,\s]','',x)))

In [93]:
#세일가 없는 행은 본래 가격이랑 동일하게
idx = (df.세일_가격=='FALSE')|(df.세일_가격=='false')|(df.세일_가격.isna())|(df.세일_가격==False)
df.loc[idx,'세일_가격'] = df.loc[idx,'가격']

#세일가격 텍스트->숫자
idx = ~df['세일_가격'].apply(lambda x: isinstance(x, int))   #텍스트값
df.loc[idx,'세일_가격'] = df.loc[idx,'세일_가격'].apply(lambda x: int(re.sub(r'[원,\s]','',x)))

In [158]:
#기획 세트 삭제
idx = df['제품명'].str.contains('기획', na=False)
df = df[~idx]
df.reset_index(drop=True, inplace=True)

In [179]:
# 용량 정리
df['용량_new'] = df['용량'].str.extract(r'(\d+\s?(ml|mL|g|G|l|L|ML|Ml))')[0]
idx = df['용량'].str.match(r'^\d+$')
df.loc[idx,'용량_new'] = df.loc[idx,'용량']
df['용량_new'] = df['용량_new'].str.replace(r'\s+', '')

df = df[~df.용량_new.isna()]   #용량정보 없는 것 삭제

array(['', '   ,\n    / ( )  .', '     () ,    ()()', 'FALSE', '  ',
       'nan', '()', ' ', '   ()()', '     ()'], dtype=object)

In [185]:
df.제조국.unique()

array(['프랑스', nan, '대한민국', '■태국', '일본', '이탈리아', '영국', '■ 독일', '한국',
       '■대한민국', '태국', '■ 대한민국', '스페인', '■대만', '미국', '홍콩', '캐나다',
       '■ 제조국 : 한국', '독일', '벨기에', '■ 미국', '국내', '이스라엘', '■말레이시아', '폴란드',
       '■ 태국', '■ 한국', '중국', '■ 일본', '■한국', '터키', '■ 스페인', '■ 중국', '불가리아',
       '■ 제조국 : 중국', '멕시코', '■ 프랑스', 'South Africa', '인도',
       '[일반 배송]\n배송지역 : 전국\n배송비 : 2,500원\n올리브영 배송 상품의 총 결제금액 20,000원 이상일 경우 무료배송 됩니다.\n도서, 산간, 오지 일부 지역은 배송비가 추가될 수 있습니다.\n배송가능일 : 3일\n배송가능일이란 본 상품을 주문하신 고객님들께 상품 배송이 가능한 기간을 의미합니다. 단, 연휴 및 공휴일은 기간 계산시 제외하며 현금 주문일 경우 입금일 기준 입니다.\n예약 상품의 경우 예약된 날짜에 출고되며, 상품의 입고가 빠르게 진행된 경우 예약일 보다 일찍 배송될 수 있습니다.\n\n[오늘드림 배송]\n배송지역 : 전국(일부지역 제외)\n배송비 : 빠름 5,000원, 3!4!/미드나잇 2,500원\n오늘드림 배송 상품의 총 결제금액 30,000원 이상일 경우 무료배송 됩니다.\n오늘드림 배송비는 도서, 산간지역에 관련없이 배송비가 동일합니다.\n배송가능시간 :\n빠름/미드나잇 당일 20시까지 주문 시 당일 도착, 이후 주문 건 익일 도착\n3!4! 당일13시까지 주문 시 당일 도착, 이후 주문 건 익일 도착',
       'Korea', '남아프리카공화국', '■이탈리아', '필리핀', '■ 말레이시아', '영국,미국', '일본/미국',
       '그리스', '■ 폴란드', 'korea', '뉴질랜드', '스위스'

#### 제품 용기 추천
- 당장 가지고 있는 데이터가 너무 없어서 제대로 된 추천x
- 추후 성능 조정 필요

In [242]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2829 entries, 0 to 2856
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   P_ID    2829 non-null   int64 
 1   카테고리_1  2829 non-null   object
 2   카테고리_2  2056 non-null   object
 3   카테고리_3  2056 non-null   object
 4   브랜드명    2827 non-null   object
 5   가격      2829 non-null   int64 
 6   class   2829 non-null   object
 7   용량_new  2829 non-null   object
dtypes: int64(2), object(6)
memory usage: 198.9+ KB


In [253]:
model_df = df[['P_ID','카테고리_1','카테고리_2','카테고리_3',
               '브랜드명','가격','class','용량_new']]

In [1]:
## 용량 단위 제거, L는 *1000해서 환산

def extract_numeric_part(text):
    match = re.search(r'(\d+)(?=\s?(ml|Ml|ML|mL|G|g)\b)', str(text), flags=re.IGNORECASE)
    if match: return match.group(1)
    else: return None
model_df['용량'] = model_df['용량_new'].apply(extract_numeric_part)

idx = (model_df['용량'].isna()) & (model_df['용량_new'].str.match(r'^\d+$'))
model_df.loc[idx, '용량'] = model_df.loc[idx, '용량_new']

idx = (model_df['용량'].isna()) & (model_df['용량_new'].str.match(r'\d+(l|L)'))
model_df.loc[idx, '용량'] = model_df[idx]['용량_new'].apply(lambda x: 
                                int(re.match(r'(\d+)(l|L)', x).group(1)) * 1000)

model_df['용량'] = model_df['용량'].astype(int)

In [257]:
#nan값 제거
# model_df = model_df.dropna()

#원핫 인코딩
# model_df = model_df.assign(**pd.get_dummies(model_df['카테고리_1']))
# model_df = model_df.assign(**pd.get_dummies(model_df['카테고리_2']))
# model_df = model_df.assign(**pd.get_dummies(model_df['카테고리_3']))
# model_df = model_df.assign(**pd.get_dummies(model_df['브랜드명']))

#레이블 인코딩
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# model_df['카테고리_1'] = label_encoder.fit_transform(model_df['카테고리_1'])
# model_df['카테고리_2'] = label_encoder.fit_transform(model_df['카테고리_2'])
# model_df['카테고리_3'] = label_encoder.fit_transform(model_df['카테고리_3'])

#드랍
# model_df.drop(['P_ID','브랜드명','용량_new'], axis=1, inplace=True)
model_df

,카테고리_1,카테고리_2,카테고리_3,가격,class,용량,AHC,CEO 보떼,CKD,CNP,...,헤라,헤이네이처,헤이미쉬,헤지스 맨 룰429,홀리추얼,홀리카홀리카,후르디아,휘게,히즈클린,힐링버드
0,7,8,19,25000,1,150,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6,13,23,28000,1,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,3,5,21000,1,500,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,6,15,46000,4,100,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,12,9,22000,1,500,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2852,6,13,23,18000,1,500,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2853,6,13,23,34000,1,45,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2854,5,4,10,9900,4,80,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2855,6,13,23,34000,2,150,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [258]:
model_df['class'] = model_df['class'].astype(int)

X = model_df.drop('class', axis=1)
y = model_df['class']

In [252]:
X[['가격','용량']]

Index(['가격', 'AHC', 'CEO 보떼', 'CKD', 'CNP', 'EGF올로지', 'OOTD', 'RMK', 'SRB',
       'owndo',
       ...
       '헤이네이처', '헤이미쉬', '헤지스 맨 룰429', '홀리추얼', '홀리카홀리카', '후르디아', '휘게', '히즈클린',
       '힐링버드', '용량'],
      dtype='object', length=425)

In [261]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                    test_size=0.3, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.5877502944640753
              precision    recall  f1-score   support

           1       0.60      0.94      0.73       466
           2       0.17      0.01      0.02        86
           3       0.00      0.00      0.00        42
           4       0.51      0.24      0.33       255

    accuracy                           0.59       849
   macro avg       0.32      0.30      0.27       849
weighted avg       0.50      0.59      0.50       849



D:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use